## <font color="green">**Biomarket s.a.s**</font>
Management software for Biomarket shop

In [25]:
# Import required packages

import json
import os

In [27]:
# Product class definition

class Product:
    """
    Represents a product in the inventory.

    Attributes:
        name (str): Name of the product.
        quantity (int): Available quantity of the product.
        purchase_price (float): Purchase price per unit.
        selling_price (float): Selling price per unit.
    """
    def __init__(self, name: str, quantity: int, purchase_price: float, selling_price: float):
        """Class constructor"""
        self.name = name
        self.quantity = quantity
        self.purchase_price = purchase_price
        self.selling_price = selling_price

    def to_dict(self) -> dict:
        """Converts the product data into a dictionary format for saving."""
        return {
            "name": self.name,
            "quantity": self.quantity,
            "purchase_price": self.purchase_price,
            "selling_price": self.selling_price
        }



In [28]:
# Definition of the BioMarket Class

class BioMarketManager:
    """
    Manages the inventory, sales, and profit calculations for BioMarket.

    Attributes:
        products (list): List of products available in the inventory.
        sales (list): List of sales transactions.
    """
    def __init__(self, product_file="products.json", sales_file="sales.json"):
        """Constructor for BioMarket, loads data from the current directory files if available."""
        self.product_file = product_file
        self.sales_file = sales_file
        self.products = self.load_data(self.product_file)
        self.sales = self.load_data(self.sales_file)

    def save_data(self, file_path: str, data):
        """Saves data to a JSON file."""
        with open(file_path, "w") as file:
            json.dump(data, file, indent=4)

    def load_data(self, file_path: str):
        """Loads data from a JSON file or returns an empty list if the file does not exist."""
        if os.path.exists(file_path):
            with open(file_path, "r") as file:
                return json.load(file)
        return []

    def add_product(self) -> None:
        """Adds a new product to the inventory or updates the quantity if the product already exists."""
        name = str(input("Product name: "))
        try:
            quantity = int(input("Enter quantity: "))
            if quantity <= 0:
                raise ValueError("Quantity must be a positive integer.")
        except ValueError as e:
            print(f"Error: {e}")
            return

        product_exists = False
        for product in self.products:
            if product["name"] == name:
                product_exists = True
                product["quantity"] += quantity
                print(f"Quantity of '{name}' updated to {product['quantity']}.")
                break

        if not product_exists:
            try:
                selling_price = float(input("Enter selling price: "))
                purchase_price = float(input("Enter purchase price: "))
            except ValueError:
                print("Error: Prices must be valid numbers.")
                return

            new_product = Product(name, quantity, purchase_price, selling_price)
            self.products.append(new_product.to_dict())
            print(f"ADDED: {quantity} X {name}")

        self.save_data(self.product_file, self.products)

    def list_products(self) -> None:
        """Lists all products in the inventory."""
        # the following command is needed to take the last updated version of the product file
        self.products = self.load_data(self.product_file)
        if not self.products:
            print("No products in inventory.")
        else:
            print("Inventory:")
            print('{:<20}{:<10}{:<18}{:<15}'.format(*['PRODUCT', "QUANTITY", 'PURCHASE PRICE', 'SELLING PRICE']))
            for product in self.products:
                print(f"{product['name']:<20}{product['quantity']:<10}€{product['purchase_price']:<17}€{product['selling_price']:<15}")

    def record_sale(self) -> None:
        """Records a sales transaction, updates product quantities, and calculates profits."""
        add_more = 'yes'
        display_string = ''
        total_sale = 0

        while add_more == 'yes':
            name = str(input("Product name: "))
            try:
                quantity_sold = int(input("Quantity: "))
                if quantity_sold <= 0:
                    raise ValueError("Quantity sold must be a positive integer.")
            except ValueError as e:
                print(f"Error: {e}")
                return

            product_found = False
            for product in self.products:
                if product["name"] == name:
                    product_found = True
                    if quantity_sold > product["quantity"]:
                        print(f"Error: Insufficient quantity for '{name}'. Available: {product['quantity']}.")
                        break

                    product["quantity"] -= quantity_sold
                    gross_profit = quantity_sold * product["selling_price"]
                    total_sale += gross_profit
                    net_profit = gross_profit - (quantity_sold * product["purchase_price"])

                    if product["quantity"] == 0:
                        self.products.remove(product)

                    self.sales.append({
                        "name": name,
                        "quantity_sold": quantity_sold,
                        "gross_profit": gross_profit,
                        "net_profit": net_profit
                    })

                    self.save_data(self.product_file, self.products)
                    self.save_data(self.sales_file, self.sales)

                    display_string += f"- {quantity_sold} X {name}: €{product['selling_price']}\n"
                    break

            if not product_found:
                print("Error: Product not found in inventory.")

            add_more = str(input("Add another product? (yes/no): "))
            while add_more not in ['yes', 'no']:
                print("Please enter 'yes' or 'no'.")
                add_more = str(input("Add another product? (yes/no): "))

        print("SALE RECORDED")
        print(display_string)
        print(f"Total: €{total_sale:.2f}")

    def show_profits(self) -> None:
        """Calculates and displays total gross and net profits from sales."""
        total_gross = sum(sale["gross_profit"] for sale in self.sales)
        total_net = sum(sale["net_profit"] for sale in self.sales)
        print(f"Total gross profit: €{total_gross:.2f}")
        print(f"Total net profit: €{total_net:.2f}")

    def help(self) -> None:
        """Displays a help menu with available commands."""
        print("Available commands:")
        print("add: Add a product to the inventory")
        print("list: List products in inventory")
        print("sell: Record a sale")
        print("profits: Show total profits (gross and net)")
        print("help: Show available commands")
        print("exit: Exit the program")

    def run(self) -> None:
        """Runs the main loop of the program, showing the menu and handling user commands."""
        while True:
            command = str(input("\nEnter a command: "))
            if command == "add":
                self.add_product()
            elif command == "list":
                self.list_products()
            elif command == "sell":
                self.record_sale()
            elif command == "profits":
                self.show_profits()
            elif command == "help":
                self.help()
            elif command == "exit":
                print("Exiting program, goodbye!")
                break
            else:
                print("Invalid command. Please try again.")
                self.help()



In [29]:
# Program execution

manage_bio_market = BioMarketManager()
manage_bio_market.run()


Enter a command: add
Product name: spinaci
Enter quantity: 20
Enter selling price: 3.5
Enter purchase price: 2.79
ADDED: 20 X spinaci

Enter a command: list
Inventory:
PRODUCT             QUANTITY  PURCHASE PRICE    SELLING PRICE  
latte di soia       25        €0.8              €1.4            
seitan              5         €2.0              €3.0            
mozzarella          7         €2.69             €4.0            
spinaci             20        €2.79             €3.5            

Enter a command: sell
Product name: mozzarella
Quantity: 7
Add another product? (yes/no): no
SALE RECORDED
- 7 X mozzarella: €4.0

Total: €28.00

Enter a command: profits
Total gross profit: €131.35
Total net profit: €53.45

Enter a command: list
Inventory:
PRODUCT             QUANTITY  PURCHASE PRICE    SELLING PRICE  
latte di soia       25        €0.8              €1.4            
seitan              5         €2.0              €3.0            
spinaci             20        €2.79             €3.5  